<a href="https://colab.research.google.com/github/sonmh79/VRPPD-with-robots/blob/main/Delivery_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time
import random

In [2]:
e1 = pd.read_excel('/content/drive/MyDrive/medium_sized_210718.xlsx',sheet_name='seed7',header=None)

In [3]:
e1.loc[9][0]

'dd =  {1: 2, 2: 1, 3: 2, 4: 1, 5: 2, 6: 2, 7: 2, 8: 2, 9: 1, 10: 2, 11: 1, 12: 2, 13: 1, 14: 2, 15: 1, 16: 1, 17: 1, 18: 1, 19: 2, 20: 1, 21: 1, 22: 1, 23: 2, 24: 2, 25: 1, 26: 1, 27: 2}'

In [4]:
customers = int(e1.loc[2][1])
customers

27

In [5]:
dd =  {1: 2, 2: 1, 3: 2, 4: 1, 5: 2, 6: 2, 7: 2, 8: 2, 9: 1, 10: 2, 11: 1, 12: 2, 13: 1, 14: 2, 15: 1, 16: 1, 17: 1, 18: 1, 19: 2, 20: 1, 21: 1, 22: 1, 23: 2, 24: 2, 25: 1, 26: 1, 27: 2}
df = pd.DataFrame(dd,index=["Delivery demand"]).T
df

,Delivery demand
1,2
2,1
3,2
4,1
5,2
6,2
7,2
8,2
9,1
10,2


In [6]:
d_data = []
data = []
for d in list(e1.loc[15:15+(customers+1)*(customers)][0]):
  data.append(float(d.split(":")[1].rstrip(",").rstrip("}")))
  if len(data) == customers:
    d_data.append(data)
    data = []

for i in range(len(d_data)):
  d_data[i].insert(i,0)
  
d_data = pd.DataFrame(d_data)
d_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.00,71.55,51.83,83.83,34.83,89.55,136.55,134.69,189.69,168.55,172.69,166.55,170.55,155.69,167.55,196.69,202.55,203.83,232.69,240.55,221.83,214.69,244.69,235.83,252.69,226.55,223.83,220.83
1,71.55,0.00,91.55,47.55,92.55,128.69,177.83,146.55,140.55,134.69,133.55,204.83,197.69,182.55,149.69,188.69,176.69,205.55,203.69,235.55,243.83,223.55,231.69,246.83,203.55,198.69,268.55,222.83
2,51.83,91.55,0.00,68.83,31.55,139.55,140.55,175.83,141.69,130.55,201.69,154.83,152.55,174.55,186.83,209.69,185.55,238.83,231.69,252.55,252.55,240.83,201.83,241.69,202.83,200.69,270.83,267.55
3,83.83,47.55,68.83,0.00,26.83,139.69,111.55,137.55,152.69,190.83,131.83,176.83,177.69,197.83,193.83,211.55,167.55,173.83,167.69,246.83,245.55,189.55,216.69,271.83,254.83,220.69,258.69,267.83
4,34.83,92.55,31.55,26.83,0.00,144.55,151.69,163.55,195.69,135.55,160.69,158.69,199.69,159.69,182.55,166.55,240.83,214.55,180.83,246.69,187.83,190.69,247.69,249.69,201.55,250.55,232.55,226.55
5,89.55,128.69,139.55,139.69,144.55,0.00,147.55,104.55,135.83,123.69,173.69,201.69,138.83,195.69,129.69,200.83,197.69,145.55,187.83,188.69,217.55,181.55,181.55,207.83,210.83,188.55,204.69,238.55
6,136.55,177.83,140.55,111.55,151.69,147.55,0.00,60.69,135.83,98.55,128.69,156.55,165.55,146.69,117.55,171.83,155.55,194.55,154.55,219.55,142.55,178.83,205.83,181.55,206.69,234.55,218.69,223.83
7,134.69,146.55,175.83,137.55,163.55,104.55,60.69,0.00,106.83,87.55,152.55,157.69,119.69,135.83,150.55,170.83,125.55,178.83,202.83,187.83,214.55,196.83,151.69,221.69,215.83,226.55,213.55,199.83
8,189.69,140.55,141.69,152.69,195.69,135.83,135.83,106.83,0.00,79.69,77.69,157.69,119.69,132.69,128.69,165.69,157.55,174.83,121.69,195.69,125.55,131.83,157.69,172.83,205.83,158.83,175.55,205.83
9,168.55,134.69,130.55,190.83,135.55,123.69,98.55,87.55,79.69,0.00,66.69,87.55,116.69,117.69,129.55,179.55,146.69,105.69,102.55,162.55,174.69,138.55,159.55,217.69,161.55,148.69,144.55,216.55


In [15]:
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class VRP:
  def __init__(self,dataset,d_data = d_data,capacity=3,cnum=100,mutation_prob=0.2,ev_times=100,robots=3,initial_input=list(range(1,len(d_data)))):
    self.n = len(d_data)
    self.c = capacity
    self.data = dataset
    self.d_data = d_data
    self.cnum = cnum
    self.cnt = 0
    self.mutation_prob = mutation_prob
    self.ev_times = ev_times
    self.dist = 0
    self.robots = robots
    self.min_route = []
    self.info = []
    self.initial_input = initial_input

  def _chromo(self,t=1):

    """ 1. Make Random Chromos """

    res = []
    for _ in range(t):
     res.append(random.sample(range(1,self.n),self.n-1)) 
    return res

  
  def calc_trips(self,trips):
    dist_list = []
    for trip in trips:
      d = 0
      for i in range(len(trip)-1):
        d += self.d_data.loc[trip[i]][trip[i+1]]
      dist_list.append(d)
    
    res = list(zip(dist_list,trips))
    res.sort()
    return res
  
  def split_trip(self, trip):

    """ Split Trip to Trips """

    stack = 0
    trips = []
    t = [0]
    for node in trip:
      if node == 0:
        stack +=1
      else:
        t.append(node)
      if stack == 2:
        stack = 1
        t.append(0)
        trips.append(t)
        t = [0]
    return trips
  
  def show_trip(self,route):

    """ Print the Trip of Min Route """

    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]
    return new_trips


  def _calc_route(self,route):

    """ 2-1-1 Calculate Distance Considering D Demands """
    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]

    res = 0
    for i in range(len(new_trips)-1):
      res += self.d_data.loc[new_trips[i]][new_trips[i+1]]
    return res

  def _getdist(self, chromos):

    """ 2-1 Return Distance of Each Chromo """

    dist_list = []

    for chromo in chromos:
      dist_list.append(self._calc_route(chromo))

    fit = list(zip(dist_list,chromos))
    fit.sort(key = lambda x:x[0])
    return fit

  def _rand(self,x,y):
    return int(random.uniform(x,y))

  def _crossover(self,p1,p2,hard_mode=False):

    """ 2-2-2. Crossover and Mutation """

    swap_point = self._rand(1,len(p1))
    c1,c2 = [],[]

    i = 0
    while i < swap_point:
      c1.append(p1[i])
      c2.append(p2[i])
      i+=1

    for e in p2:
      if e not in c1:
        c1.append(e)

    for e in p1:
      if e not in c2:
        c2.append(e)
    
    #mutation
    if random.uniform(0,1) <= self.mutation_prob and len(c1)>2 and not hard_mode:
      e1,e2 = random.sample(range(1,len(c1)),2)
      #target will select mutation child
      if e1 > e2:
        target = e1
      else:
        target = e2
      
      if target%2 == 0:
        c1[e1],c1[e2] = c1[e2],c1[e1]
      else:  
        c2[e1],c2[e2] = c2[e2],c2[e1]

    if hard_mode:
      e1,e2 = random.sample(range(0,len(c1)-1),2)
      c1[e1],c1[e1+1] = c1[e1+1],c1[e1]
      c2[e2],c2[e2+1] = c2[e2+1],c2[e2]

    next_gen = self._getdist([c1,c2,p1,p2])
    #print("next_gen: ",next_gen)
    c1,c2 = next_gen[0][1],next_gen[1][1]
    #print("c1,c2: ",c1,c2)
    return [c1,c2]

  
  def _select_parent(self,parents):

    """ 2-2-1. Select Parents with Roulette Algorithm  """

    fitness = [1/route[0] for route in parents] 
    sum = 0

    for f in fitness:
      sum += f
    p_index = set()

    while len(p_index)<2:
      fit = 0
      target = random.uniform(0,sum)
      for i in range(len(fitness)):
        fit += fitness[i]
        if fit > target:
          p_index.add(i)
          break  
    return list(p_index)

  
  def _make_child(self,dist_routes):

    """ 2-2. Compare Parents with Childs and Return Best Routes  """

    self.cnt += 1
    parents = dist_routes.copy()
    routes = [route[1] for route in parents]
    child = []

    # select top 100 chromos
    while parents and len(child)<self.cnum:
      selected_parents = self._select_parent(parents)
      p1,p2 = routes.pop(selected_parents[0]),routes.pop(selected_parents[1]-1)
      parents.pop(selected_parents[0]),parents.pop(selected_parents[1]-1)
      c1,c2 = self._crossover(p1,p2)
      child += c1,c2

    return child

  def evolution(self,chromo):

    """ 2. Make Childs """
    
    dist_routes = self._getdist(chromo) #(dist,routes)
    child = self._make_child(dist_routes)
    return child

  def ga(self): 

    """ 0. Start Genetic Algorithm !! """

    chromos = self._chromo(99)
    chromos.append(self.initial_input)    
    gen = self.evolution(chromos)
    self.info = self._getdist(gen)
    print("First Genenration Minimun Route: ",(self.info[0][0],self.info[0][1]))
    for i in range(self.ev_times):
      gen = self.evolution(gen)
      self.info = self._getdist(gen)
      t = self.cnt
      if t % 10 == 0:
        print("{}/{}th Evolution Min Route: ".format(t,self.ev_times),(self.info[0][0],self.info[0][1]))
      
      # to avoid local minimum
      if t == 30:
        self.mutation_prob = t/100
        print(f"==================== Mutation Prob Upgrade to {self.mutation_prob} ==================== ")
      elif t == 50:
        self.mutation_prob = t/100
        print(f"==================== Mutation Prob Upgrade to {self.mutation_prob} ==================== ")
      elif t == 70:
        self.mutation_prob = t/100
        print(f"==================== Mutation Prob Upgrade to {self.mutation_prob} ==================== ")
      
    self.dist,self.min_route = self.info[0][0],self.info[0][1]   
    print("----------------------------------------Finish----------------------------------------")
    print("Total Travel Time / Min Route: ",self.dist," / ",self.min_route)
    print(self.show_trip(self.info[0][1]))
#if __name__ == "__main__":
  


In [14]:
# Medium Data Seed7
v = VRP(df,d_data,capacity=3,ev_times=30,robots=3)
s= time.time()
v.ga()
trips = v.split_trip(v.show_trip(v.info[0][1]))
trips_info = v.calc_trips(trips)
robot = [[] for _ in range(v.robots)]
score = [0 for _ in range(v.robots)] 
for info in trips_info:
  index = score.index(min(score))
  score[index] += info[0]
  robot[index].append(info[1])

print("Robot1: ",robot[0], "Time: ", score[0])
print("Robot2: ",robot[1], "Time: ", score[1])
print("Robot3: ",robot[2], "Time: ", score[2])
e = time.time()
print("Total Time: ",round(e-s,1),"(s)")

First Genenration Minimun Distance:  (6217.14, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])
10/30th Evolution Min Route:  (5960.139999999999, [4, 3, 25, 23, 20, 10, 1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 26, 27])
20/30th Evolution Min Route:  (5780.14, [1, 2, 3, 4, 5, 6, 10, 7, 8, 9, 11, 12, 13, 14, 17, 15, 16, 18, 19, 20, 21, 22, 25, 23, 24, 26, 27])
30/30th Evolution Min Route:  (5754.31, [4, 3, 25, 23, 20, 10, 1, 2, 8, 18, 5, 6, 7, 9, 11, 12, 13, 14, 17, 15, 16, 19, 21, 22, 27, 24, 26])
==================== Mutation Prob Upgrade to 0.3 ====================: 
----------------------------------------Finish----------------------------------------
Total Travel Time / Min Route:  5751.31  /  [1, 2, 4, 3, 25, 23, 20, 10, 8, 18, 5, 6, 7, 9, 11, 12, 13, 14, 17, 15, 16, 19, 22, 21, 27, 24, 26]
[0, 1, 2, 0, 4, 3, 0, 25, 23, 0, 20, 10, 0, 8, 18, 0, 5, 0, 6, 0, 7, 9, 0, 11, 12, 0, 13, 14, 0, 17, 15, 16, 0, 19,